In [1]:
!pip install requests python-dotenv pandas


### Import Required Libraries and Set Up Environment Variables

In [2]:
# Dependencies
import requests
import time
from dotenv import load_dotenv
import os
import pandas as pd
import json

In [3]:
# Set environment variables from the .env in the local environment
load_dotenv("example.env")

nyt_api_key = os.getenv("NYT_API_KEY")
tmdb_api_key = os.getenv("TMDB_API_KEY")
print(nyt_api_key)
print(tmdb_api_key)

6WR5AxO2pg2FfB7dG7O9lYmNBgmckJYz
0cdde7e48ec8a4f6609e36c1fc1d0035


### Access the New York Times API

In [4]:
# Set the base URL
url = "https://api.nytimes.com/svc/search/v2/articlesearch.json?"

# Filter for movie reviews with "love" in the headline
# section_name should be "Movies"
# type_of_material should be "Review"
filter_query = 'section_name:"Movies" AND type_of_material:"Review" AND headline:"love"'

# Use a sort filter, sort by newest
sort = "newest"

# Select the following fields to return:
# headline, web_url, snippet, source, keywords, pub_date, byline, word_count
field_list = "headline,web_url,snippet,source,keywords,pub_date,byline,word_count"

# Search for reviews published between a begin and end date
begin_date = "20130101"
end_date = "20230531"

# Build URL
constructed_url = f"{url}fq={filter_query}&sort={sort}&fl={field_list}&begin_date={begin_date}&end_date={end_date}"
print(constructed_url)

https://api.nytimes.com/svc/search/v2/articlesearch.json?fq=section_name:"Movies" AND type_of_material:"Review" AND headline:"love"&sort=newest&fl=headline,web_url,snippet,source,keywords,pub_date,byline,word_count&begin_date=20130101&end_date=20230531


In [5]:
# Create an empty list to store the reviews
reviews_list = []

# loop through pages 0-19
for page in range(20):
    # create query with a page number
    # API results show 10 articles at a time
    page_query = f"&page={page}"

    # Make a "GET" request and retrieve the JSON
    response = requests.get(constructed_url + page_query, params={"api-key": nyt_api_key})

    # Add a twelve-second interval between queries to stay within API query limits
    time.sleep(12)

    # Try and save the reviews to the reviews_list
    try:
        reviews = response.json()

        # loop through the reviews["response"]["docs"] and append each review to the list
        for review in reviews["response"]["docs"]:
            reviews_list.append(review)

        # Print the page that was just retrieved
        print(f"Checked page {page}")

    except Exception as e:
        # Print the page number that had no results then break from the loop
        print(f"No results found for page {page + 1}: {e}")
        break



Checked page 0
Checked page 1
Checked page 2
Checked page 3
Checked page 4
Checked page 5
Checked page 6
Checked page 7
Checked page 8
Checked page 9
Checked page 10
Checked page 11
Checked page 12
Checked page 13
Checked page 14
Checked page 15
Checked page 16
Checked page 17
Checked page 18
Checked page 19


In [6]:
# Preview the first 5 results in JSON format
# Use json.dumps with argument indent=4 to format data
print(json.dumps(reviews_list[:5], indent=4))


[
    {
        "web_url": "https://www.nytimes.com/2023/05/25/movies/the-attachment-diaries-review.html",
        "snippet": "A gynecologist and her patient form a horrifyingly twisted connection in this batty, bloody Argentine melodrama.",
        "source": "The New York Times",
        "headline": {
            "main": "\u2018The Attachment Diaries\u2019 Review: Love, Sick",
            "kicker": null,
            "content_kicker": null,
            "print_headline": "The Attachment Diaries",
            "name": null,
            "seo": null,
            "sub": null
        },
        "keywords": [
            {
                "name": "subject",
                "value": "Movies",
                "rank": 1,
                "major": "N"
            },
            {
                "name": "creative_works",
                "value": "The Attachment Diaries (Movie)",
                "rank": 2,
                "major": "N"
            },
            {
                "name": "persons",
 

In [7]:
# Convert reviews_list to a Pandas DataFrame using json_normalize()
df_reviews = pd.json_normalize(reviews_list)
print(df_reviews)

                                               web_url  \
0    https://www.nytimes.com/2023/05/25/movies/the-...   
1    https://www.nytimes.com/2023/05/04/movies/what...   
2    https://www.nytimes.com/2023/05/04/movies/you-...   
3    https://www.nytimes.com/2023/04/21/movies/a-to...   
4    https://www.nytimes.com/2023/04/20/movies/othe...   
..                                                 ...   
195  https://www.nytimes.com/2017/03/09/movies/the-...   
196  https://www.nytimes.com/2017/03/09/movies/revi...   
197  https://www.nytimes.com/2017/03/02/movies/love...   
198  https://www.nytimes.com/2017/02/16/movies/ever...   
199  https://www.nytimes.com/2017/02/09/movies/kedi...   

                                               snippet              source  \
0    A gynecologist and her patient form a horrifyi...  The New York Times   
1    Two childhood friends navigate cultural differ...  The New York Times   
2    Religion comes between two girls falling in lo...  The New York 

In [8]:
# Extract the title from the "headline.main" column and
# save it to a new column "title"
# Title is between unicode characters \u2018 and \u2019.
# End string should include " Review" to avoid cutting title early

# Extract the title from the "headline.main" column
def extract_title(headline):
    start_index = headline.find('\u2018') + len('\u2018')
    end_index = headline.find(' Review') + len(' Review')
    return headline[start_index:end_index]

# Apply the extract_title function to create the new "title" column
df_reviews['title'] = df_reviews['headline.main'].apply(extract_title)

In [9]:
# Extract 'name' and 'value' from items in "keywords" column
def extract_keywords(keyword_list):
    extracted_keywords = ""
    for item in keyword_list:
        # Extract 'name' and 'value'
        keyword = f"{item['name']}: {item['value']};"
        # Append the keyword item to the extracted_keywords list
        extracted_keywords += keyword
    return extracted_keywords

# Fix the "keywords" column by converting cells from a list to a string
df_reviews['keywords'] = df_reviews['keywords'].apply(extract_keywords)


In [10]:
# Create a list from the "title" column using to_list()
title_list = df_reviews['title'].to_list()
title_list = [title.replace('’ Review', '').strip() for title in title_list]

# These titles will be used in the query for The Movie Database
# Print each title from title_list on a separate line
for title in title_list:
    print(title)

The Attachment Diaries

You Can Live Forever
A Tourist’s Guide to Love
Other People’s Children
One True Loves
The Lost Weekend: A Love Story
A Thousand and One
Your Place or Mine
Love in the Time of Fentanyl
Pamela, a Love Story
In From the Side
After Love
Alcarràs
Nelly & Nadine
Lady Chatterley’s Lover
The Sound of Christmas
The Inspection
Bones and All
My Policeman
About Fate
Waiting for Bojangles
I Love My Dad
A Love Song
Alone Together
Art of Love
The Wheel
Thor: Love and Thunder
Both Sides of the Blade
Fire of Love
Love & Gelato
Stay Prayed Up
Benediction
Dinner in America
In a New York Minute
Anaïs in Love
I Love America
See You Then
La Mami
Love After Love
Deep Water
Lucy and Desi
Cyrano
The In Between
Book of Love
Lingui, the Sacred Bonds
The Pink Cloud
A Journal for Jordan
West Side Story
Aulcie
Love Is Love Is Love
Love Hard
Bergman Island
Hard Luck Love Song
South of Heaven
Wife of a Spy
Happier Than Ever
Together
Annette
Resort to Love
Woodstock 99: Peace, Love and Rage
Cas

### Access The Movie Database API

In [11]:
# Prepare The Movie Database query
url = "https://api.themoviedb.org/3/search/movie?query="
tmdb_key_string = "&api_key=" + tmdb_api_key

In [12]:
# Create an empty list to store the results
tmdb_movies_list = []


# Create an empty list to store the results
tmdb_movies_list = []

# Create a request counter to sleep the requests after a multiple of 50 requests
request_counter = 0

# Loop through the titles
for title in title_list:
    # Check if we need to sleep before making a request
    if request_counter % 50 == 0 and request_counter != 0:
        time.sleep(10)  # Sleep for 10 seconds after every 50 requests

    # Add 1 to the request counter
    request_counter += 1

    # Perform a "GET" request for The Movie Database
    response = requests.get(url + title + tmdb_key_string)

    # Include a try clause to search for the full movie details.
    # Use the except clause to print out a statement if a movie is not found.
    try:
        # Get movie id
        movie_id = response.json()["results"][0]["id"]

        # Make a request for the full movie details
        movie_details_url = f"https://api.themoviedb.org/3/movie/{movie_id}?api_key={tmdb_api_key}"
        movie_details_response = requests.get(movie_details_url)

        # Execute "GET" request with url
        movie_details = movie_details_response.json()

        # Extract the genre names into a list
        genres = [genre["name"] for genre in movie_details["genres"]]

        # Extract the spoken_languages' English name into a list
        spoken_languages = [lang["english_name"] for lang in movie_details["spoken_languages"]]

        # Extract the production_countries' name into a list
        production_countries = [country["name"] for country in movie_details["production_countries"]]

        # Add the relevant data to a dictionary and append it to the tmdb_movies_list list
        tmdb_movies_list.append({
            "title": movie_details["title"],
            "overview": movie_details["overview"],
            "genres": genres,
            "spoken_languages": spoken_languages,
            "production_countries": production_countries
        })

        # Print out the title that was found
        print(f"Found {title}")

    except IndexError:
        print(f"Movie not found: {title}")

# Print the tmdb_movies_list
for movie in tmdb_movies_list:
    print(movie)



Found The Attachment Diaries
Movie not found: 
Found You Can Live Forever
Found A Tourist’s Guide to Love
Found Other People’s Children
Found One True Loves
Found The Lost Weekend: A Love Story
Found A Thousand and One
Found Your Place or Mine
Found Love in the Time of Fentanyl
Found Pamela, a Love Story
Found In From the Side
Found After Love
Found Alcarràs
Found Nelly & Nadine
Found Lady Chatterley’s Lover
Found The Sound of Christmas
Found The Inspection
Found Bones and All
Found My Policeman
Found About Fate
Found Waiting for Bojangles
Found I Love My Dad
Found A Love Song
Found Alone Together
Found Art of Love
Found The Wheel
Found Thor: Love and Thunder
Found Both Sides of the Blade
Found Fire of Love
Found Love & Gelato
Found Stay Prayed Up
Found Benediction
Found Dinner in America
Found In a New York Minute
Found Anaïs in Love
Found I Love America
Found See You Then
Found La Mami
Found Love After Love
Found Deep Water
Found Lucy and Desi
Found Cyrano
Found The In Between
Found 

In [13]:
# Preview the first 5 results in JSON format
# Use json.dumps with argument indent=4 to format data
print(json.dumps(tmdb_movies_list[:5], indent=4))

[
    {
        "title": "The Attachment Diaries",
        "overview": "Argentina, 1970s. A desperate young woman goes to a clinic to have a clandestine abortion. As her pregnancy is already through the fourth month, the doctor refuses. Instead, she proposes to sell the baby to one of her clients and offers to provide shelter in her house until the child is born. Their disturbed personalities will become intertwined in a strange and dangerous relationship.",
        "genres": [
            "Drama",
            "Mystery",
            "Thriller",
            "Horror"
        ],
        "spoken_languages": [
            "Spanish"
        ],
        "production_countries": [
            "Argentina"
        ]
    },
    {
        "title": "You Can Live Forever",
        "overview": "When Jaime, a gay teenager, is sent to live in a community of Jehovah's Witnesses in small-town Quebec, she falls madly in love with a devout Witness girl. The two embark on an intense affair with consequences t

In [14]:
# Convert the results to a DataFrame table
df_tmdb_movies = pd.DataFrame(tmdb_movies_list)

# Print the DataFrame
df_tmdb_movies

,title,overview,genres,spoken_languages,production_countries
0,The Attachment Diaries,"Argentina, 1970s. A desperate young woman goes...","[Drama, Mystery, Thriller, Horror]",[Spanish],[Argentina]
1,You Can Live Forever,"When Jaime, a gay teenager, is sent to live in...","[Drama, Romance]","[English, French]","[Canada, United States of America]"
2,A Tourist's Guide to Love,"After an unexpected break up, a travel executi...","[Romance, Comedy]","[English, Vietnamese]",[United States of America]
3,Other People's Children,"Rachel loves her life, her students, her frien...","[Drama, Comedy]","[French, English]",[France]
4,One True Loves,Emma and Jesse are living the perfect life tog...,"[Romance, Comedy, Drama]","[English, Spanish]","[Czech Republic, United States of America]"
...,...,...,...,...,...
136,Sicilian Ghost Story,In a little Sicilian village at the edge of a ...,"[Mystery, Drama, Fantasy]",[Italian],"[France, Italy, Switzerland]"
137,The New Romantic,In order to escape her looming post-graduation...,"[Drama, Comedy]",[English],[Canada]
138,Pimp,An urban love story set on the hard streets of...,[Drama],[English],[United States of America]
139,In a Relationship,Long-term couple Owen and Hallie are breaking ...,"[Drama, Comedy, Romance]",[English],"[Mexico, United States of America]"


### Merge and Clean the Data for Export

In [15]:
# Merge the New York Times reviews and TMDB DataFrames on title
merged_df = pd.merge(df_reviews, df_tmdb_movies, on="title", how="inner")

# Print the merged DataFrame
merged_df

,web_url,snippet,source,keywords,pub_date,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,...,headline.seo,headline.sub,byline.original,byline.person,byline.organization,title,overview,genres,spoken_languages,production_countries
0,https://www.nytimes.com/2017/06/22/movies/my-j...,"In “My Journey Through French Cinema,” Mr. Tav...",The New York Times,subject: Documentary Films and Programs;person...,2017-06-22T21:14:56+0000,773,"Review: Those Movies, Himself — Bertrand Taver...",None,None,Reflecting on a Lifelong Love,...,None,None,By Manohla Dargis,"[{'firstname': 'Manohla', 'middlename': None, ...",None,Review,A young woman recounts a story to a group of f...,[],[English],[]


In [16]:
# Remove list brackets and quotation marks on the columns containing lists
# Define characters to remove (adjust as per your requirements)
chars_to_remove = ['[', ']', '\'']

# Create a list of the columns that need fixing
columns_to_fix = ['genres', 'spoken_languages', 'production_countries']


# Loop through the list of columns to fix
for column in columns_to_fix:
    # Convert the column to type 'str'
    merged_df[column] = merged_df[column].astype(str)

    # Loop through characters to remove
    for char in chars_to_remove:
        merged_df[column] = merged_df[column].str.replace(char, '')

# Display the fixed DataFrame
print(merged_df)


                                             web_url  \
0  https://www.nytimes.com/2017/06/22/movies/my-j...   

                                             snippet              source  \
0  In “My Journey Through French Cinema,” Mr. Tav...  The New York Times   

                                            keywords  \
0  subject: Documentary Films and Programs;person...   

                   pub_date  word_count  \
0  2017-06-22T21:14:56+0000         773   

                                       headline.main headline.kicker  \
0  Review: Those Movies, Himself — Bertrand Taver...            None   

  headline.content_kicker        headline.print_headline  ... headline.seo  \
0                    None  Reflecting on a Lifelong Love  ...         None   

  headline.sub    byline.original  \
0         None  By Manohla Dargis   

                                       byline.person byline.organization  \
0  [{'firstname': 'Manohla', 'middlename': None, ...                None   

    

In [17]:
# Drop "byline.person" column
merged_df.drop(columns='byline.person', inplace=True)


In [18]:
# Delete duplicate rows
merged_df.drop_duplicates(inplace=True)

# Reset index
merged_df.reset_index(drop=True, inplace=True)

# Print the DataFrame after deleting duplicates and resetting index
print(merged_df)


                                             web_url  \
0  https://www.nytimes.com/2017/06/22/movies/my-j...   

                                             snippet              source  \
0  In “My Journey Through French Cinema,” Mr. Tav...  The New York Times   

                                            keywords  \
0  subject: Documentary Films and Programs;person...   

                   pub_date  word_count  \
0  2017-06-22T21:14:56+0000         773   

                                       headline.main headline.kicker  \
0  Review: Those Movies, Himself — Bertrand Taver...            None   

  headline.content_kicker        headline.print_headline headline.name  \
0                    None  Reflecting on a Lifelong Love          None   

  headline.seo headline.sub    byline.original byline.organization   title  \
0         None         None  By Manohla Dargis                None  Review   

                                            overview genres spoken_languages  \
0  

In [19]:
# Export data to CSV without the index
merged_df.to_csv('merged_movie_data.csv', index=False)